In [1]:
from langchain_community.llms import Tongyi
import sys
sys.path.insert(0, '../')
from common.utils import Utils

In [2]:
import os
os.environ["DASHSCOPE_API_KEY"] = Utils.get_tongyi_key()
#llm = Tongyi(model_name= 'qwen-max-1201', temperature = 0)
from common.CustomLLM import QwenLLM
url = "https://dashscope.aliyuncs.com/api/v1/services/aigc/text-generation/generation"
token = Utils.get_tongyi_key()

qwen = QwenLLM(url= url, model='qwen-max-1201', token=token, temperature=0.2)

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "/Users/collin/.cache/modelscope/hub/AI-ModelScope/bge-base-zh-v1.5"

encode_kwargs = {'normalize_embeddings' : True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs,
    query_instruction = "为这个句子生成表示用于检索相关文章：",#"Represent this sentence for searching relevant passages:"
)

In [20]:
import time
start = time.time()
embedding_model.embed_query("""Dear Alumni,

We would like to invite you to take part in the University’s website user feedback questionnaire. Your thoughts and opinions are highly valued as crucial feedback for ongoing efforts to enhance the website. We appreciate your time and assure you that all responses will be kept strictly confidential.

The questionnaire should take approximately 3 minutes to complete and can be accessed through the link below. As a token of our gratitude, the first 100 participants will receive a silicone travel tag as a reward. An email will be sent to winners and details of the redemption will be provided on 30 Jan 2024. Please kindly complete the questionnaire before 12 Jan 2024.""")
print(time.time() - start)

0.12381625175476074


In [14]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine.sub_question_query_engine import SubQuestionQueryEngineCustom
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext

In [11]:
service_context = ServiceContext.from_defaults(llm = qwen, embed_model = embedding_model, chunk_size=2048, chunk_overlap= 200)

### Created Index

In [12]:
dbs_report = SimpleDirectoryReader(input_files = ['./data/230327-annual-report-and-accounts-2022-cn.pdf']).load_data()#input_dir="./data/paul_graham/"

/Users/collin/miniconda3/envs/llm/lib/python3.9/site-packages/pypdf/_cmap.py:183: PdfReadWarning: Advanced encoding /ETenms-B5-H not implemented yet
  warnings.warn(
/Users/collin/miniconda3/envs/llm/lib/python3.9/site-packages/pypdf/_cmap.py:183: PdfReadWarning: Advanced encoding /UniCNS-UTF16-H not implemented yet
  warnings.warn(


In [15]:
# build index and query engine
vector_dbs_idnex = VectorStoreIndex.from_documents(
    dbs_report, service_context=service_context
)

vector_dbs_idnex.storage_context.persist(persist_dir = './data/subquery/hsbc')


### Load Index

In [16]:
from llama_index import StorageContext, load_index_from_storage

storage_dbs_doc = StorageContext.from_defaults(persist_dir = './data/subquery/hsbc')
storage_dbs_index = load_index_from_storage(storage_dbs_doc, service_context=service_context)

In [24]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=storage_dbs_index.as_query_engine(),
        metadata=ToolMetadata(
            name="滙豐控股有限公司2022年報及賬目",
            description="提供滙豐控股有限公司2022年財報的信息"
        ),
    ),
]

query_engine = SubQuestionQueryEngineCustom.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    verbose=True,
    use_async=False,
)

In [31]:
response = query_engine.query('策略表現指標')


In [32]:
for chunk in response:
    print(chunk)
    print('-' * 5)

Generated 1 sub questions.
Generated 1 Sub Questions.

-----
[滙豐控股有限公司2022年報及賬目] Q: 請問貴公司今年的策略表現為何？
[滙豐控股有限公司2022年報及賬目] Q: 請問貴公司今年的策略表現為何？

-----
[滙豐控股有限公司2022年報及賬目] A:  對不起，我找不到答案
[滙豐控股有限公司2022年報及賬目] A:  對不起，我找不到答案

-----
 根據提供的上下文，無法確定「策略表現」指標的具體定義或衡量標準。若要獲得更詳細的答案，建議向該公司詢問其評估策略表現的方式和相關數據。
-----


In [13]:
#!pip install llama-index==0.9.15.post2 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [14]:
s = 'abd'
s.replace('a', '')
s

'abd'

In [ ]:
import asyncio
import logging
from typing import List, Optional, Sequence, cast

from llama_index.async_utils import run_async_tasks
from llama_index.bridge.pydantic import BaseModel, Field
from llama_index.callbacks.base import CallbackManager
from llama_index.callbacks.schema import CBEventType, EventPayload
from llama_index.core import BaseQueryEngine
from llama_index.prompts.mixin import PromptMixinType
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.openai_generator import OpenAIQuestionGenerator
from llama_index.question_gen.types import BaseQuestionGenerator, SubQuestion
from llama_index.response.schema import RESPONSE_TYPE
from llama_index.response_synthesizers import BaseSynthesizer, get_response_synthesizer
from llama_index.schema import NodeWithScore, QueryBundle, TextNode
from llama_index.service_context import ServiceContext
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.utils import get_color_mapping, print_text

logger = logging.getLogger(__name__)


class SubQuestionAnswerPair(BaseModel):
    """
    Pair of the sub question and optionally its answer (if its been answered yet).
    """

    sub_q: SubQuestion
    answer: Optional[str] = None
    sources: List[NodeWithScore] = Field(default_factory=list)


class SubQuestionQueryEngineCustom(BaseQueryEngine):
    """Sub question query engine.

    A query engine that breaks down a complex query (e.g. compare and contrast) into
        many sub questions and their target query engine for execution.
        After executing all sub questions, all responses are gathered and sent to
        response synthesizer to produce the final response.

    Args:
        question_gen (BaseQuestionGenerator): A module for generating sub questions
            given a complex question and tools.
        response_synthesizer (BaseSynthesizer): A response synthesizer for
            generating the final response
        query_engine_tools (Sequence[QueryEngineTool]): Tools to answer the
            sub questions.
        verbose (bool): whether to print intermediate questions and answers.
            Defaults to True
        use_async (bool): whether to execute the sub questions with asyncio.
            Defaults to True
    """

    def __init__(
        self,
        question_gen: BaseQuestionGenerator,
        response_synthesizer: BaseSynthesizer,
        query_engine_tools: Sequence[QueryEngineTool],
        callback_manager: Optional[CallbackManager] = None,
        verbose: bool = True,
        use_async: bool = False,
    ) -> None:
        self._question_gen = question_gen
        self._response_synthesizer = response_synthesizer
        self._metadatas = [x.metadata for x in query_engine_tools]
        self._query_engines = {
            tool.metadata.name: tool.query_engine for tool in query_engine_tools
        }
        self._verbose = verbose
        self._use_async = use_async
        super().__init__(callback_manager)

    def _get_prompt_modules(self) -> PromptMixinType:
        """Get prompt sub-modules."""
        return {
            "question_gen": self._question_gen,
            "response_synthesizer": self._response_synthesizer,
        }

    @classmethod
    def from_defaults(
        cls,
        query_engine_tools: Sequence[QueryEngineTool],
        question_gen: Optional[BaseQuestionGenerator] = None,
        response_synthesizer: Optional[BaseSynthesizer] = None,
        service_context: Optional[ServiceContext] = None,
        verbose: bool = True,
        use_async: bool = True,
    ) -> "SubQuestionQueryEngine":
        callback_manager = None
        if service_context is not None:
            callback_manager = service_context.callback_manager
        elif len(query_engine_tools) > 0:
            callback_manager = query_engine_tools[0].query_engine.callback_manager

        if question_gen is None:
            if service_context is None:
                # use default openai model that supports function calling API
                question_gen = OpenAIQuestionGenerator.from_defaults()
            else:
                # try to use OpenAI function calling based question generator.
                # if incompatible, use general LLM question generator
                try:
                    question_gen = OpenAIQuestionGenerator.from_defaults(
                        llm=service_context.llm
                    )
                except ValueError:
                    question_gen = LLMQuestionGenerator.from_defaults(
                        service_context=service_context
                    )

        synth = response_synthesizer or get_response_synthesizer(
            callback_manager=callback_manager,
            service_context=service_context,
            use_async=use_async,
        )

        return cls(
            question_gen,
            synth,
            query_engine_tools,
            callback_manager=callback_manager,
            verbose=verbose,
            use_async=use_async,
        )

    def _query(self, query_bundle: QueryBundle) -> RESPONSE_TYPE:
        with self.callback_manager.event(
            CBEventType.QUERY, payload={EventPayload.QUERY_STR: query_bundle.query_str}
        ) as query_event:
            sub_questions = self._question_gen.generate(self._metadatas, query_bundle)

            colors = get_color_mapping([str(i) for i in range(len(sub_questions))])

            if self._verbose:
                print_text(f"Generated {len(sub_questions)} sub questions.\n")
            yield f"Generated {len(sub_questions)} Sub Questions.\n"

            if self._use_async:
                tasks = [
                    self._aquery_subq(sub_q, color=colors[str(ind)])
                    for ind, sub_q in enumerate(sub_questions)
                ]

                qa_pairs_all = run_async_tasks(tasks)
                qa_pairs_all = cast(List[Optional[SubQuestionAnswerPair]], qa_pairs_all)
            else:
                qa_pairs_all = []
                #    self._query_subq(sub_q, color=colors[str(ind)])
                #    for ind, sub_q in enumerate(sub_questions)
                #]
                #print("Question", sub_questions)
                for ind, sub_q in enumerate(sub_questions):
                    for item in self._query_subq(sub_q, color=colors[str(ind)]):
                        #print(ind, item)
                        if isinstance(item, str):
                            yield item
                        else:
                            #print('itemss')
                            qa_pairs_all.append(item)
                            break

            # filter out sub questions that failed
            qa_pairs: List[SubQuestionAnswerPair] = list(filter(None, qa_pairs_all))

            nodes = [self._construct_node(pair) for pair in qa_pairs]

            source_nodes = [node for qa_pair in qa_pairs for node in qa_pair.sources]
            response = self._response_synthesizer.synthesize(
                query=query_bundle,
                nodes=nodes,
                additional_source_nodes=source_nodes,
            )

            query_event.on_end(payload={EventPayload.RESPONSE: response})
        yield response
        #return response

    async def _aquery(self, query_bundle: QueryBundle) -> RESPONSE_TYPE:
        with self.callback_manager.event(
            CBEventType.QUERY, payload={EventPayload.QUERY_STR: query_bundle.query_str}
        ) as query_event:
            sub_questions = await self._question_gen.agenerate(
                self._metadatas, query_bundle
            )

            colors = get_color_mapping([str(i) for i in range(len(sub_questions))])

            if self._verbose:
                print_text(f"Generated {len(sub_questions)} sub questions.\n")

            tasks = [
                self._aquery_subq(sub_q, color=colors[str(ind)])
                for ind, sub_q in enumerate(sub_questions)
            ]

            qa_pairs_all = await asyncio.gather(*tasks)
            qa_pairs_all = cast(List[Optional[SubQuestionAnswerPair]], qa_pairs_all)

            # filter out sub questions that failed
            qa_pairs: List[SubQuestionAnswerPair] = list(filter(None, qa_pairs_all))

            nodes = [self._construct_node(pair) for pair in qa_pairs]

            source_nodes = [node for qa_pair in qa_pairs for node in qa_pair.sources]
            response = await self._response_synthesizer.asynthesize(
                query=query_bundle,
                nodes=nodes,
                additional_source_nodes=source_nodes,
            )

            query_event.on_end(payload={EventPayload.RESPONSE: response})

        return response

    def _construct_node(self, qa_pair: SubQuestionAnswerPair) -> NodeWithScore:
        node_text = (
            f"Sub question: {qa_pair.sub_q.sub_question}\nResponse: {qa_pair.answer}"
        )
        return NodeWithScore(node=TextNode(text=node_text))

    async def _aquery_subq(
        self, sub_q: SubQuestion, color: Optional[str] = None
    ) -> Optional[SubQuestionAnswerPair]:
        try:
            with self.callback_manager.event(
                CBEventType.SUB_QUESTION,
                payload={EventPayload.SUB_QUESTION: SubQuestionAnswerPair(sub_q=sub_q)},
            ) as event:
                question = sub_q.sub_question
                query_engine = self._query_engines[sub_q.tool_name]

                if self._verbose:
                    print_text(f"[{sub_q.tool_name}] Q: {question}\n", color=color)

                response = await query_engine.aquery(question)
                response_text = str(response)

                if self._verbose:
                    print_text(f"[{sub_q.tool_name}] A: {response_text}\n", color=color)

                qa_pair = SubQuestionAnswerPair(
                    sub_q=sub_q, answer=response_text, sources=response.source_nodes
                )

                event.on_end(payload={EventPayload.SUB_QUESTION: qa_pair})

            return qa_pair
        except ValueError:
            logger.warning(f"[{sub_q.tool_name}] Failed to run {question}")
            return None

    def _query_subq(
        self, sub_q: SubQuestion, color: Optional[str] = None
    ):
        try:
            question = sub_q.sub_question
            query_engine = self._query_engines[sub_q.tool_name]
            if self._verbose:
                print_text(f"[{sub_q.tool_name}] Q: {question}\n", color=color)
            yield f"[{sub_q.tool_name}] Q: {question}\n"
            response = query_engine.query(question)
            response_text = str(response)
            if self._verbose:
                print_text(f"[{sub_q.tool_name}] A: {response_text}\n", color=color)
            yield f"[{sub_q.tool_name}] A: {response_text}\n"#f"[{sub_q.tool_name}] A: {response_text}\n"
            qa_pair = SubQuestionAnswerPair(
                sub_q=sub_q, answer=response_text, sources=response.source_nodes
            )
            #event.on_end(payload={EventPayload.SUB_QUESTION: qa_pair})
            yield qa_pair
            #print('test 1')
            #return qa_pair
        except ValueError:
            #print('eeror')
            logger.warning(f"[{sub_q.tool_name}] Failed to run {question}")
            #yield None
            #return None


In [6]:
d = {'d' : 1}
print(list(d.values())[0])

1


In [8]:
if d.get('a'):
    print(d)

In [14]:
''.join(['','']) == ''

True

In [20]:
s = ['tsst']
print('\n'.join(s[:-1]))

In [21]:
'\n'.join(s[:-1])

''